<a href="https://colab.research.google.com/github/nehaal10/Cloud_deployment/blob/main/personal_loan_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from imblearn.combine import SMOTEENN 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
import pickle
import joblib

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/project/Bank_Personal_Loan_Modelling.csv')
df.drop(['ID','ZIP Code'],axis=1,inplace=True)

In [ ]:
cols=list(df.columns)

In [ ]:
minm=MinMaxScaler()
df[cols]=minm.fit_transform(df[cols])

In [64]:
from sklearn.pipeline import make_pipeline

pipelines = {
    'lg':make_pipeline(LogisticRegression(random_state=1234))
}

In [68]:
x=df.drop('Personal Loan',axis=1)
y=df['Personal Loan']

sme=SMOTEENN(random_state=42)
x_res,y_res=sme.fit_resample(x,y)
xtrain,xtest,ytrain,ytest=train_test_split(x_res,y_res,shuffle=True,test_size=0.2,random_state=42)

In [ ]:
lg_columns=list(xtrain.columns)
joblib.dump(lg_columns, '/content/drive/MyDrive/kaggle/lg_columns.pkl')

['/content/drive/MyDrive/kaggle/lg_columns.pkl']

In [39]:
lrgreg=LogisticRegression(solver='lbfgs')
lrgreg.fit(xtrain,ytrain)

LogisticRegression()

In [40]:
ypred=lrgreg.predict(xtest)

In [41]:
accuracy_score(ytest,ypred)

0.9273140261215219

In [42]:
with open ('/content/drive/MyDrive/project/logreg_personal_loan.pkl','wb') as f:
  pickle.dump(lrgreg,f)

In [43]:
!pip install -U ibm-watson-machine-learning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [44]:
from ibm_watson_machine_learning import APIClient
import json
import numpy as np

In [48]:
vml_credential={
    'apikey':'gwUVQsmIWTA2wWdBeU7q-wK4Kdb87qG5Q-bFHZyg5RdD',
    'url':"https://us-south.ml.cloud.ibm.com"
}

In [49]:
wml_client = APIClient(vml_credential)
wml_client.spaces.list()

Python 3.7 and 3.8 frameworks are deprecated and will be removed in a future release. Use Python 3.9 framework instead.
Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  ------------  ------------------------
ID                                    NAME          CREATED
86d23858-2e24-419f-9290-f652894d2272  Personalloan  2022-09-07T22:09:51.952Z
------------------------------------  ------------  ------------------------


In [50]:
space_id='86d23858-2e24-419f-9290-f652894d2272'
wml_client.set.default_space(space_id)

'SUCCESS'

In [51]:
MODEL_NAME = 'LogisticRegression'
DEPLOYMENT_NAME = 'ML Deployment'
BEST_MODEL = lrgreg

In [57]:
xtrain=xtrain.reset_index()
xtrain.drop('index',axis=1,inplace=True)

In [58]:
ytrain=xtrain.reset_index()
ytrain.drop('index',axis=1,inplace=True)

In [79]:
# Set Python Version
software_spec_uid = wml_client.software_specifications.get_id_by_name('runtime-22.1-py3.9')

# Setup model meta
model_props = {
    wml_client.repository.ModelMetaNames.NAME: MODEL_NAME, 
    wml_client.repository.ModelMetaNames.TYPE: 'scikit-learn_1.0', 
    wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid 
}

#Save model
model_details = wml_client.repository.store_model(model=BEST_MODEL, meta_props=model_props, training_data=None)


In [82]:
model_uid = wml_client.repository.get_model_id(model_details)

In [83]:
model_uid

'07a53cb8-5c71-4af4-87c5-29cb5c7dc153'